In [1]:
from docx import Document
import re


In [2]:
def load_docx(file_path):
    try:
        return Document(file_path)
    except Exception as e:
        print(f"Error loading document: {e}")
        return None

doc = load_docx('data/vb_test.docx')
if not doc:
    exit()


In [3]:
import string
seps = '[.,;-+]'
ascii = string.ascii_lowercase
lowest_level = [(i + ")") for i in ascii]
levels = [lowest_level]

In [4]:
def extract_text(doc):
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                full_text.append(cell.text)
    return '\n'.join(full_text)

document_text = extract_text(doc)


In [5]:
alphabet_lst = [i for i in ascii]
digit_lst = [str(i) for i in range(99)]
bullet_levels1 = [["chương"], ["điều"], digit_lst, alphabet_lst]
bullet_levels2 = [["chương"], ["phụ lục"], digit_lst, alphabet_lst]

In [6]:
from collections import OrderedDict

In [7]:
text = []
tree = OrderedDict()
for i, para in enumerate(doc.paragraphs):
    text.append(para.text)

In [8]:
full_text = []
c_check = False
for i, para in enumerate(text):
    if c_check == True:
        c_check = False
        continue
    if para.split(" ")[0].lower() == "chương":
        para = para + " " + text[i + 1]
        c_check = True
    else:
        c_check = False
    full_text.append(para)

In [9]:
def update_tree(bullet, par):
    for i in range (4):
        if bullet.lower() in bullet_levels1[i]:
            k = tree
            for j in range(i):
                k = k[next(reversed(k))]
            k[par] = OrderedDict()

In [10]:
for i, para in enumerate(full_text):
    first_token = para.split(" ")[0]
    if len(first_token.strip()) > 0:
        bullet = re.split(r"[.,;)]",first_token)[0]
        update_tree(bullet, para)


In [11]:
def flatten_tree(tree, parent_path="", separator=" > "):
    flat_list = []
    for key, value in tree.items():
        current_path = f"{parent_path}{separator}{key}" if parent_path else key
        # print(f"Processing: {current_path}")  # Debugging step
        if value:  # Check if the value is not blank OrderedDict
            flat_list.extend(flatten_tree(value, current_path, separator))
        else:
            # If value is a blank ordered dict, convert value to blank string
            flat_list.append((current_path, ""))
    return flat_list

In [12]:
flat_data = flatten_tree(tree)
chunks = [f"{path}: {text}" for path, text in flat_data]


In [13]:
from langchain.vectorstores import FAISS
from langchain.embeddings import GPT4AllEmbeddings
from typing import List, Dict, Tuple
# Initialize the embedding model
embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")

# Embed and store in FAISS
db = FAISS.from_texts(texts=chunks, embedding=embedding_model)

# Save the FAISS vector database
vector_db_path = "vectorstores/db_faiss"
db.save_local(vector_db_path)

Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
